# 월간분석 기능

예시 데이터

```python
{
  "date": "2025-12-01",
  "user_name": "김현빈",
  "birth_date": "2000-05-05",
  "gender": "남성",
  "emotion_score": 5,
  "dream_content": "어젯밤 꿈에서 나는 끝없이 펼쳐진 황금빛 들판을 걷고 있었습니다. 하늘은 맑고 푸르렀으며 바람이 부드럽게 불어왔습니다. 들판 한가운데에는 거대한 나무가 서 있었고, 그 나무 아래에는 반짝이는 보물상자가 놓여 있었습니다. 상자를 열자 그 안에는 빛나는 황금 열쇠가 들어 있었습니다. 열쇠를 손에 쥐는 순간 주변의 모든 것이 더욱 밝게 빛나기 시작했습니다. 나는 이 열쇠로 어딘가의 문을 열어야 한다는 확신이 들었습니다.",
  "title": "김현빈님의 2025-12-01 통합 운세",
  "dream_interpretation": "새로운 기회의 문이 열리는 길조몽으로, 재물운과 사업운이 상승하는 시기입니다.",
  "today_fortune_summary": "오늘의 꿈(황금 들판과 보물 열쇠)은 새로운 시작과 기회를 상징하는 매우 길한 꿈입니다. 황금빛 들판은 풍요와 번영을, 보물상자는 숨겨진 가치의 발견을, 열쇠는 문제 해결의 실마리를 의미합니다. 12월의 시작과 함께 새로운 프로젝트나 계획이 순조롭게 진행될 것을 암시하며, 특히 재정적인 측면에서 긍정적인 변화가 예상됩니다. 황소자리 기질의 김현빈님에게는 실용적이고 안정적인 성과를 얻을 수 있는 기회가 찾아올 것입니다.",
  "action_guide": "1) 새로운 프로젝트나 계획을 시작하기 좋은 날입니다: 미뤄왔던 일이 있다면 오늘부터 첫 발걸음을 떼세요.\n2) 재정 계획을 검토하세요: 투자나 저축 계획을 세우기에 적합한 시기이니 재무 상태를 점검하세요.\n3) 중요한 결정은 오전 중에: 해가 높을 때 판단력이 가장 명확하므로 중요한 선택은 오전에 하세요.\n4) 네트워킹에 집중하세요: 새로운 인연이나 협력 관계가 열쇠가 될 수 있으니 적극적으로 소통하세요.\n5) 황금색 소품을 활용하세요: 명함지갑이나 펜 등 황금색 아이템을 사용하면 운기를 더욱 높일 수 있습니다.\n6) 감사 일기를 작성하세요: 저녁에 오늘의 긍정적인 일들을 기록하며 에너지를 축적하세요.",
  "lucky_color": {
    "name": "황금색",
    "reason": "황금색은 풍요와 성공을 상징하며, 꿈에서 본 황금 들판과 열쇠의 기운을 현실로 가져옵니다. 5월생 황소자리인 김현빈님에게 물질적 안정과 실질적 성과를 가져다주는 색입니다."
  },
  "lucky_item": {
    "name": "황동 열쇠고리",
    "reason": "꿈에서 얻은 황금 열쇠의 상징성을 일상에서 구현할 수 있는 아이템입니다. 황동 소재의 열쇠고리를 가방이나 지갑에 달아두면 새로운 기회를 여는 통로가 되어주며, 실용성을 중시하는 황소자리 성향과도 잘 맞습니다."
  }
}
```
를 기준으로

사용하는 데이터는 아래와 같다.
dream_content 
today_fortune_summary 
emotion_score
date

In [7]:
def create_compact_monthly_prompt(data_folder='./test_dataset/december'):
    """31일 전체를 압축된 형태로 프롬프트에 포함"""
    
    # 데이터 로드
    all_data = []
    for file_path in sorted(glob.glob(f'{data_folder}/*.json')):
        with open(file_path, 'r', encoding='utf-8') as f:
            all_data.append(json.load(f))
    
    # 통계
    emotion_scores = [d['emotion_score'] for d in all_data]
    stats = {
        'total_days': len(all_data),
        'avg_emotion': round(sum(emotion_scores) / len(emotion_scores), 2),
        'emotion_distribution': {i: emotion_scores.count(i) for i in range(1, 6)},
        'period': all_data[0]['date'][:7],
        'user_name': all_data[0]['user_name'],
        'birth_date': all_data[0]['birth_date']
    }
    
    emotion_labels = {
        1: "매우 부정적", 2: "부정적", 3: "중립", 4: "긍정적", 5: "매우 긍정적"
    }
    
    prompt = f"""당신은 꿈 분석 전문가입니다. {stats['user_name']}님({stats['birth_date']}생)의 {stats['period']} 한 달 꿈 일기를 분석해주세요.

## 통계
- 총 {stats['total_days']}일 기록
- 평균 감정: {stats['avg_emotion']}/5.0
- 분포: 😊{stats['emotion_distribution'][5]}일 🙂{stats['emotion_distribution'][4]}일 😐{stats['emotion_distribution'][3]}일 😟{stats['emotion_distribution'][2]}일 😢{stats['emotion_distribution'][1]}일

## 날짜별 운세 분석 (전체 31일)

"""
    
    # 31일 전체를 간결하게 포함
    for data in all_data:
        # today_fortune_summary의 핵심만 추출 (첫 2문장 또는 200자)
        summary = data['today_fortune_summary']
        # 마침표나 느낌표 기준으로 앞 2문장만
        sentences = summary.replace('다. ', '다.|').replace('니다. ', '니다.|').split('|')
        compact_summary = sentences[0] if sentences else summary[:200]
        
        prompt += f"**[{data['date']}] 감정{data['emotion_score']}점** - {data['dream_interpretation']}\n"
        prompt += f"{compact_summary}\n\n"
    
    prompt += f"""
## 작성 요청

위 31일 전체의 흐름을 분석하여 감동적인 월간 리포트를 작성해주세요:

### 1. 이번 달의 감정 여정 (300-400자)
한 달간의 감정 기복과 흐름을 서정적으로. 초반/중반/후반의 변화를 포함.

### 2. 꿈이 전하는 메시지 (400-500자)
반복되는 주제, 패턴, 무의식의 메시지를 구체적 날짜와 함께.

### 3. 심리적 통찰과 성장 (300-400자)
내면의 변화와 성장의 증거들.

### 4. 다음 달을 위한 조언 (200-250자)
따뜻하고 실천 가능한 격려.

**스타일**: 감동적, 개인화, 서정적. {stats['user_name']}님을 자연스럽게 호칭.

각 섹션 구분은 명확하되 흐름은 자연스럽게하고 사용자에겐 목차를 보여주지 마세요.
"""

    return prompt, stats

In [8]:
async def generate_monthly_report_with_prompt(prompt, stats):
    """주어진 프롬프트로 월간 리포트 생성 (스트리밍)"""
    
    print("=" * 60)
    print("🌙 월간 꿈 분석 리포트 생성 시작")
    print("=" * 60)
    
    print(f"\n프롬프트 정보:")
    print(f"   - 길이: {len(prompt):,}자")
    print(f"   - 예상 토큰: ~{len(prompt)//4:,} tokens")
    print(f"   - 기간: {stats['period']}")
    print(f"   - 사용자: {stats['user_name']}님")
    
    print("\n[API 호출 중] 실시간 스트리밍...")
    print("=" * 60)
    print()
    
    monthly_report = ""
    
    try:
        stream = await client.chat.completions.create(
            model='gpt-4.1',
            messages=[
                {
                    "role": "system", 
                    "content": "당신은 꿈과 심리 분석 전문가입니다. 감동적이고 서정적인 글을 작성합니다."
                },
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
            max_tokens=2500,
            temperature=0.8,
            stream=True,
        )
        
        async for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                monthly_report += content
                print(content, end="", flush=True)
        
        print("\n")
        print("=" * 60)
        print("\n✅ 생성 완료!")
        
        # 결과 저장
        output = {
            'period': stats['period'],
            'user_name': stats['user_name'],
            'statistics': stats,
            'report': monthly_report,
            'metadata': {
                'total_days': stats['total_days'],
                'prompt_length': len(prompt),
                'avg_emotion': stats['avg_emotion']
            }
        }
        
        # JSON 파일로 저장
        output_file = f"monthly_report_{stats['period']}.json"
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(output, f, ensure_ascii=False, indent=2)
        
        print(f"\n💾 결과가 '{output_file}'에 저장되었습니다.")
        
        return output
        
    except Exception as e:
        print(f"\n❌ 오류 발생: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

In [10]:
import pandas as pd
import glob
import json

prompt1, stats = create_compact_monthly_prompt()
result = await generate_monthly_report_with_prompt(prompt1, stats)

🌙 월간 꿈 분석 리포트 생성 시작

프롬프트 정보:
   - 길이: 4,312자
   - 예상 토큰: ~1,078 tokens
   - 기간: 2025-12
   - 사용자: 김현빈님

[API 호출 중] 실시간 스트리밍...


❌ 오류 발생: name 'client' is not defined


Traceback (most recent call last):
  File "/var/folders/_2/vwg7t2kd5193tdlmvxrzrmk80000gn/T/ipykernel_61610/3398385988.py", line 21, in generate_monthly_report_with_prompt
    stream = await client.chat.completions.create(
                   ^^^^^^
NameError: name 'client' is not defined


In [14]:
import json
import glob
from pathlib import Path

def merge_december_data_for_monthly_analysis(
    input_folder='./test_dataset/december',
    output_file='./test_dataset/monthly_analysis_request_december.json'  # 📝 파일명 지정
):
    """
    december 폴더의 모든 JSON 파일을 월간 분석 API 요청 형식으로 병합
    
    Args:
        input_folder: JSON 파일들이 있는 폴더 경로
        output_file: 생성할 병합 파일 경로
    """
    
    # 모든 JSON 파일 읽기 (정렬된 순서로)
    json_files = sorted(glob.glob(f'{input_folder}/fortune_*.json'))
    
    if not json_files:
        print(f"❌ {input_folder}에서 JSON 파일을 찾을 수 없습니다.")
        return
    
    print(f"📂 발견한 파일: {len(json_files)}개")
    
    all_data = []
    user_name = None
    birth_date = None
    
    # 각 파일 읽기
    for file_path in json_files:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            
            # 첫 번째 파일에서 사용자 정보 가져오기
            if user_name is None:
                user_name = data['user_name']
                birth_date = data['birth_date']
            
            # 필요한 필드만 추출
            daily_data = {
                "date": data['date'],
                "dream_content": data['dream_content'],
                "today_fortune_summary": data['today_fortune_summary'],
                "emotion_score": data['emotion_score']
            }
            
            all_data.append(daily_data)
    
    # 월간 분석 API 요청 형식으로 구성
    monthly_request = {
        "user_name": user_name,
        "birth_date": birth_date,
        "daily_data": all_data
    }
    
    # JSON 파일로 저장
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(monthly_request, f, ensure_ascii=False, indent=2)
    
    print(f"✅ 병합 완료!")
    print(f"   - 사용자: {user_name}")
    print(f"   - 생년월일: {birth_date}")
    print(f"   - 총 일수: {len(all_data)}일")
    print(f"   - 저장 위치: {output_file}")
    
    # 통계 정보 출력
    emotion_scores = [d['emotion_score'] for d in all_data]
    avg_emotion = sum(emotion_scores) / len(emotion_scores)
    emotion_distribution = {i: emotion_scores.count(i) for i in range(1, 6)}
    
    print(f"\n📊 감정 통계:")
    print(f"   - 평균: {avg_emotion:.2f}/5.0")
    print(f"   - 분포: ", end="")
    for score in range(5, 0, -1):
        count = emotion_distribution.get(score, 0)
        emoji = ["😢", "😟", "😐", "🙂", "😊"][score-1]
        print(f"{emoji}{count}일 ", end="")
    print()
    
    return monthly_request


if __name__ == "__main__":
    # 실행
    result = merge_december_data_for_monthly_analysis()
    
    # 결과 미리보기 (첫 2개 항목만)
    if result:
        print("\n📄 생성된 JSON 미리보기 (처음 2일):")
        preview = {
            "user_name": result["user_name"],
            "birth_date": result["birth_date"],
            "daily_data": result["daily_data"][:2]  # 처음 2개만 표시
        }
        print(json.dumps(preview, ensure_ascii=False, indent=2))
        print(f"\n... (총 {len(result['daily_data'])}일)")

📂 발견한 파일: 31개
✅ 병합 완료!
   - 사용자: 김현빈
   - 생년월일: 2000-05-05
   - 총 일수: 31일
   - 저장 위치: ./test_dataset/monthly_analysis_request_december.json

📊 감정 통계:
   - 평균: 4.13/5.0
   - 분포: 😊15일 🙂9일 😐3일 😟4일 😢0일 

📄 생성된 JSON 미리보기 (처음 2일):
{
  "user_name": "김현빈",
  "birth_date": "2000-05-05",
  "daily_data": [
    {
      "date": "2025-12-01",
      "dream_content": "어젯밤 꿈에서 나는 끝없이 펼쳐진 황금빛 들판을 걷고 있었습니다. 하늘은 맑고 푸르렀으며 바람이 부드럽게 불어왔습니다. 들판 한가운데에는 거대한 나무가 서 있었고, 그 나무 아래에는 반짝이는 보물상자가 놓여 있었습니다. 상자를 열자 그 안에는 빛나는 황금 열쇠가 들어 있었습니다. 열쇠를 손에 쥐는 순간 주변의 모든 것이 더욱 밝게 빛나기 시작했습니다. 나는 이 열쇠로 어딘가의 문을 열어야 한다는 확신이 들었습니다.",
      "today_fortune_summary": "오늘의 꿈(황금 들판과 보물 열쇠)은 새로운 시작과 기회를 상징하는 매우 길한 꿈입니다. 황금빛 들판은 풍요와 번영을, 보물상자는 숨겨진 가치의 발견을, 열쇠는 문제 해결의 실마리를 의미합니다. 12월의 시작과 함께 새로운 프로젝트나 계획이 순조롭게 진행될 것을 암시하며, 특히 재정적인 측면에서 긍정적인 변화가 예상됩니다. 황소자리 기질의 김현빈님에게는 실용적이고 안정적인 성과를 얻을 수 있는 기회가 찾아올 것입니다.",
      "emotion_score": 5
    },
    {
      "date": "2025-12-02",
      "dream_content": "꿈속에서 나는 무서운 살인마에게 쫓기며 필사적으로 도망치고 있었습니다. 어두운 골목길을 

In [21]:
print = """
"인사말 — 서릿발 내려앉은 12월, 희미한 달빛이 창가에 내려앉아 조용히 속삭입니다.\n\n이번 달의 감정 여정 — 김현빈님, 12월은 당신의 감정이 파도처럼 잔잔하게 출렁이다가도 때로는 급류를 만나 다시 평온으로 돌아오는 한 달이었습니다. 초반(1~10일)은 황금 들판(12-01), 하늘을 나는 자유(12-03) 같은 환희와 함께 살인마에게 쫓기는 불안(12-02), 시험 불안(12-04), 물에 가라앉는 공포(12-06)이 교차하며 대비를 보여주었습니다. 중반(11~20일)에는 조상과의 만남(12-11), 정상의 성취(12-12), 유성우(12-15)처럼 성취와 위안이 이어졌고, 권위와 보호를 상징하는 용과 호랑이(12-07, 12-16)도 등장해 심리적 안정감을 더했습니다. 후반(21~31일)은 과수원의 풍요(12-21), 보물창고(12-23), 왕좌(12-27), 우주여행(12-29) 등 다채로운 길몽들이 연속되며 한 해를 아름답게 마무리하는 기운으로 가득했습니다. 평균 감정 4.13점과 15일의 환한 웃음이 말해주듯, 전반적으로 회복력과 기쁨이 우세한 달이었습니다.\n\n꿈이 전하는 메시지 — 이달의 꿈들은 반복적으로 “확장과 발견”을 노래합니다. 금빛 들판(12-01), 지갑과 보물(12-05, 12-23)은 숨겨진 가치와 예상치 못한 기회를 알리며, 과수원(12-21)과 보물창고는 실질적 결실을 예고합니다. 자유와 상승의 이미지(하늘을 나는 12-03, 산 정상 12-12, 용과 비행 12-16, 탑의 일출 12-25, 우주 12-29)는 당신이 한계 너머로 나아갈 준비가 되었음을 말합니다. 반면 쫓김(12-02), 시험불안(12-04), 길잃음(12-10), 물속 공포(12-06)은 준비와 통제에 대한 불안으로 나타났고, 집의 붕괴(12-08)와 다리를 건너는 장면(12-26)은 낡은 구조의 해체와 새로움으로의 이행을 상징합니다. 보호자 이미지들(호랑이 12-07, 할아버지 12-11, 고래 12-18)은 외부의 도움과 내면의 안전감을 반복적으로 확인하게 합니다. 요컨대 이 달의 메시지는 “두려움은 변화의 그림자일 뿐, 기회는 이미 당신 안과 주변에 열려 있다”입니다.\n\n심리적 통찰과 성장 — 김현빈님은 이달에 두 갈래의 내적 성장을 겪었습니다. 하나는 자기효능감의 회복으로, 무대에서의 인정(12-19), 왕좌와 권위(12-27), 성취의 상징들(12-12, 12-15)이 당신의 자신감과 사회적 위상을 강화했습니다. 다른 하나는 불안과의 화해입니다. 쫓김, 물속, 시험의 꿈들은 스트레스 반응을 알렸지만, 이후로 보호와 지지가 반복되어 나타난 것은 당신이 외부의 도움을 수용하고 안전을 재구축하는 법을 배우고 있다는 신호입니다. 또한 도서관(12-13), 일기장(12-30), 지혜의 나무(12-28)는 내적 탐구와 초심 회복의 욕구를 보여주며, 이는 단순한 승리가 아니라 의미 있는 성찰과 통합으로 이어지고 있습니다.\n\n다음 달을 위한 조언 — 따뜻한 격려를 전합니다. 풍요의 징후가 많으니 작은 계획부터 실행하세요: 한 가지 아이디어를 정해 일주일 단위로 실천하고, 불안이 올 때는 호흡과 몸의 감각으로 자리잡으세요. 도움을 주겠다는 호랑이와 할아버지의 이미지처럼 신뢰할 수 있는 사람 한 명에게 현재의 고민을 나눠보면 좋습니다. 꿈을 계속 기록하며 반복되는 상징(보물, 비상, 보호)을 현실적 목표로 전환해보세요. 겨울의 고요 속에서 천천히 다음 계절을 준비해도 됩니다."""

print

'\n"인사말 — 서릿발 내려앉은 12월, 희미한 달빛이 창가에 내려앉아 조용히 속삭입니다.\n\n이번 달의 감정 여정 — 김현빈님, 12월은 당신의 감정이 파도처럼 잔잔하게 출렁이다가도 때로는 급류를 만나 다시 평온으로 돌아오는 한 달이었습니다. 초반(1~10일)은 황금 들판(12-01), 하늘을 나는 자유(12-03) 같은 환희와 함께 살인마에게 쫓기는 불안(12-02), 시험 불안(12-04), 물에 가라앉는 공포(12-06)이 교차하며 대비를 보여주었습니다. 중반(11~20일)에는 조상과의 만남(12-11), 정상의 성취(12-12), 유성우(12-15)처럼 성취와 위안이 이어졌고, 권위와 보호를 상징하는 용과 호랑이(12-07, 12-16)도 등장해 심리적 안정감을 더했습니다. 후반(21~31일)은 과수원의 풍요(12-21), 보물창고(12-23), 왕좌(12-27), 우주여행(12-29) 등 다채로운 길몽들이 연속되며 한 해를 아름답게 마무리하는 기운으로 가득했습니다. 평균 감정 4.13점과 15일의 환한 웃음이 말해주듯, 전반적으로 회복력과 기쁨이 우세한 달이었습니다.\n\n꿈이 전하는 메시지 — 이달의 꿈들은 반복적으로 “확장과 발견”을 노래합니다. 금빛 들판(12-01), 지갑과 보물(12-05, 12-23)은 숨겨진 가치와 예상치 못한 기회를 알리며, 과수원(12-21)과 보물창고는 실질적 결실을 예고합니다. 자유와 상승의 이미지(하늘을 나는 12-03, 산 정상 12-12, 용과 비행 12-16, 탑의 일출 12-25, 우주 12-29)는 당신이 한계 너머로 나아갈 준비가 되었음을 말합니다. 반면 쫓김(12-02), 시험불안(12-04), 길잃음(12-10), 물속 공포(12-06)은 준비와 통제에 대한 불안으로 나타났고, 집의 붕괴(12-08)와 다리를 건너는 장면(12-26)은 낡은 구조의 해체와 새로움으로의 이행을 상징합니다. 보호자 이미지들(호랑이 12-07, 할아버지 12-11, 고래 12-18)은 외부의 도움과 내면의 안전감을 반